In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# --- 1. Load and Explore the Data ---
print("Loading data...")
# Make sure 'train.csv' is in the same directory as your notebook
df = pd.read_csv('train.csv')

print("--- Initial Data Exploration ---")
print("\nFirst 5 Rows:")
print(df.head())

print("\nData Info:")
df.info()

print("\nMissing Values Count:")
print(df.isnull().sum())


# --- 2. Data Cleaning and Preprocessing ---
print("\n--- Starting Data Cleaning and Preprocessing ---")

# Drop Loan_ID as it's not useful for prediction
df.drop('Loan_ID', axis=1, inplace=True)

# Impute missing values for categorical columns with the mode (most frequent value)
for col in ['Gender', 'Married', 'Dependents', 'Self_Employed']:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Impute missing Credit_History with the mode
df['Credit_History'].fillna(df['Credit_History'].mode()[0], inplace=True)

# Impute missing LoanAmount with the median to avoid outlier influence
df['LoanAmount'].fillna(df['LoanAmount'].median(), inplace=True)

# Impute missing Loan_Amount_Term with the mode
df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mode()[0], inplace=True)

print("\nMissing values after imputation (should all be zero):")
print(df.isnull().sum())

# Convert categorical columns to numerical using one-hot encoding
# drop_first=True helps avoid multicollinearity
df_processed = pd.get_dummies(df, drop_first=True)

print("\nData processed and ready for modeling.")


# --- 3. Build Prediction System ---
print("\n--- Building Prediction System ---")

# Define features (X) and target (y)
X = df_processed.drop('Loan_Status_Y', axis=1)
y = df_processed['Loan_Status_Y']

# Split data into training (80%) and testing (20%) sets
# stratify=y ensures that the proportion of classes is the same in both train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize the RandomForestClassifier
# class_weight='balanced' helps the model handle the imbalanced nature of the target variable
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# Train the model on the training data
print("Training the model...")
model.fit(X_train, y_train)
print("Model training complete.")


# --- 4. Evaluate the Model ---
print("\n--- Model Evaluation ---")

# Make predictions on the test data
y_pred = model.predict(X_test)

# Print the accuracy and a detailed classification report
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Rejected (0)', 'Approved (1)']))


# --- 5. Feature Importance ---
print("\n--- Top 10 Feature Importances ---")

# Get and display the most important features
feature_importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
print(feature_importances.head(10))

Loading data...
--- Initial Data Exploration ---

First 5 Rows:
    Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001002   Male      No          0      Graduate            No   
1  LP001003   Male     Yes          1      Graduate            No   
2  LP001005   Male     Yes          0      Graduate           Yes   
3  LP001006   Male     Yes          0  Not Graduate            No   
4  LP001008   Male      No          0      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   Credit_History Property_Area Loan_Status  
0             1.0         Urban           Y  
1 

C:\Users\shash\AppData\Local\Temp\ipykernel_2748\872050750.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
C:\Users\shash\AppData\Local\Temp\ipykernel_2748\872050750.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exampl

Model training complete.

--- Model Evaluation ---
Accuracy: 0.84

Classification Report:
              precision    recall  f1-score   support

Rejected (0)       0.82      0.61      0.70        38
Approved (1)       0.84      0.94      0.89        85

    accuracy                           0.84       123
   macro avg       0.83      0.77      0.79       123
weighted avg       0.84      0.84      0.83       123


--- Top 10 Feature Importances ---
ApplicantIncome            0.221936
LoanAmount                 0.198076
Credit_History             0.184600
CoapplicantIncome          0.121795
Loan_Amount_Term           0.047790
Property_Area_Semiurban    0.033902
Married_Yes                0.029310
Education_Not Graduate     0.028268
Dependents_1               0.027077
Property_Area_Urban        0.025189
dtype: float64
